In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
test_string = '[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}'

In [3]:
bad_string = '[.##.#..#.#] (0,1,2,4,5,6,7,8) (1,2) (1,4,5) (0,6) (0,2,6,8,9) (0,1,5,7,8) (2,4) (3,9) (1,3,4,5,6,7,8,9) {37,48,21,35,39,45,37,42,44,37}'

In [4]:
from queue import PriorityQueue
import numpy as np

from z3 import (
    Int, IntVector,
    Solver, Optimize,
    Sum, sat
)

class Machine:
    def __init__(self,line):
        elements = line.strip().split()

        self.target_state = [ int(t=='#') for t in elements[0].strip('[]')]
        self.joltage = [ int(t) for t in elements[-1].strip('{}').split(',')]
        buttons = []
        for b in elements[1:-1]:
            buttons.append( [ int(t) for t in b.strip('()').split(',')] )

        self.buttons = buttons


    def find_least_pushes(self):
        init_state = [0]*len(self.target_state)
        
        frontier = PriorityQueue() 
        frontier.put((0, init_state))
        seen = {tuple(init_state) : 0}
        
        while not frontier.empty():
            step, state = frontier.get()

            for button in self.buttons:
                new_state = state.copy()
                for b in button:
                    new_state[b] = (new_state[b] +1)%2

                if new_state == self.target_state:
                    return step+1

                if tuple(new_state) not in seen.keys() or seen[tuple(new_state)] > step+1 : 
                    seen[tuple(new_state)] =  step+1
                    frontier.put((step+1, new_state))
        return None

        
    def find_least_pushes_jotage(self):
        n_buttons = len(self.buttons)
        n_lights = len(self.joltage)


        A_t=[]
        for b in self.buttons:
            A_t.append( [ int(t in b) for t in range(n_lights)])
        A_t = np.vstack(A_t)
        A = A_t.T

        press_var = IntVector('p', n_buttons)

        s = Optimize()
        
        for p in press_var:
            s.add(p >= 0)
        
        for j in range(n_lights):
            s.add(
                Sum(A[j][i] * press_var[i] for i in range(n_buttons)) == self.joltage[j]
            )
       # Minimize total presses
        
        s.minimize(Sum( press_var))
        
        assert s.check() == sat
        model = s.model()
        
        answer = sum(model[p].as_long() for p in press_var) 
        return answer
        


In [5]:
def part_one(file_path='input_data/test_10.txt'):
    total_pushes=0
    
    with open(file_path,'r') as f:
        for line in f.readlines():
            m = Machine(line)
            steps = m.find_least_pushes()
            total_pushes+=steps
            
    return total_pushes
           

In [6]:
%%time
part_one()

CPU times: user 741 μs, sys: 1.19 ms, total: 1.93 ms
Wall time: 1.34 ms


7

In [7]:
%%time
part_one('input_data/day_10.txt')

CPU times: user 89.5 ms, sys: 3.22 ms, total: 92.7 ms
Wall time: 96.3 ms


409

# Part 2


In [8]:
tmp = Machine(test_string)

In [9]:
tmp.find_least_pushes_jotage()

10

In [10]:
def part_two(file_path='input_data/test_10.txt'):
    total_pushes=0
    
    with open(file_path,'r') as f:
        for l, line in enumerate(f.readlines()):
            m = Machine(line)
            steps = m.find_least_pushes_jotage()
            total_pushes+=steps
            
    return total_pushes
           

In [11]:
%%time
part_two()

CPU times: user 19.3 ms, sys: 2.35 ms, total: 21.7 ms
Wall time: 20.4 ms


33

In [12]:
%%time
part_two('input_data/day_10.txt')

CPU times: user 561 ms, sys: 6.79 ms, total: 568 ms
Wall time: 631 ms


15489